In [10]:
with open('test.txt', 'r') as f:
    f_contents = f.read(206)
    print(f_contents, end='')

Pontificia Universidad Catolica del Ecuador


Programa para calcular el corte en zapatas rectangulares

Autor: Nicolas Jativa
Fecha: 19/03/2023
Materia: Cimentaciones
Profesor: Carlos Celi
Tarea: numero 2



Aqui las referencias de la ACI:

<img src='./images/cortante1.jpg'>
<img src='./images/cortante2.jpg'>
<img src='./images/cortante3.jpg'>
<img src='./images/cortante4.jpg'>

<p>Referencia para el acero de refuerzo</p>
<img src='./images/refuerzo.jpg' width=300px height=300px>

In [11]:
import numpy as np
import pandas as pd



cc = 65 #cm
cl = 85 #cm
B = 165 #cm
L = 165 #cm
bw = 100 #cm
A0 = B*L #cm2
rec = 7.5 #cm

fprimc = 230 #kg/cm2
fy = 4200 #kg/cm2
phi = 0.75 
lmbda = 1 #Relación según resistencia del hormigón (ACI 22.6.5.3)
direccion = False

qa = 2 #kg/cm2
qu = 1.2*qa #kg/cm2
Pl = 12.8 * 1000 #kg
Pd = 19.2 * 1000 #kg

t = 0
if direccion == True:
    t = cl
elif direccion == False:
    t = cc

d = np.min([cc,cl]) - rec  #en cm
Psu = (1.2*Pd) + (1.6*Pl)
qsu = Psu / A0
e = B/2 - t/2 - d
beta = np.max([cc,cl])/np.min([cc,cl])


#Verificacion de espesor minimo requerido por cortante
lmbdasmat=[]
bomat = []
Cortanteactual = []
Cortante2011 = []
dval = []
Vu1mat = []
Vu2mat = []
phiVc1mat = []
phiVc2mat = []
phiVc3mat = []
phiVc2011mat = []
phiVcmaxmat = []
phiVcd2mat = []
phiVcdmat = []
ratiodmat = []
ratiod2mat = []

i=0
de=d
phiVcd2 = Vu2 =0

while phiVcd2 <= Vu2:

    alphas = 40   # The value of alphaS is 40 for interior columns, 30 for edge columns, and 20 for corner columns. (22.6.5.3)
    bo = (cc+cl+(2*de))*2 #Perímetro de la sección crítica para cortante en dos direcciones en losas y zapatas. (!!Preguntar como lo consideraa en la clase!!)
    bomat.append(bo)

    lmbdas = np.sqrt(2/(1+(0.04*de)))
    lmbdasmat.append(lmbdas)

    if lmbdas <=1:
        lmbdas = lmbdas
    else:
        lmbdas = 1


    Vu1 = 16 #qsu*e*bw / 1000 |En T @ 100cm| #Cortante Unidireccional
    Vu1mat.append(Vu1)

    Vu2 = 18 #(Psu - (qsu*((t+de)**2)))*(bw/bo) / 1000   |En T @ 100cm| #Cortante Bidireccional
    Vu2mat.append(Vu2)

    phiVc1 = phi*1.1*lmbda*lmbdas*np.sqrt(fprimc)*bw*de /1000    #ACI 22.6.5.2 (a)
    phiVc1mat.append(phiVc1)
    
    phiVc2 = phi*0.53*(1+(2/beta))*lmbdas*lmbda*np.sqrt(fprimc)*bw*de /1000    #ACI 22.6.5.2 (b)
    phiVc2mat.append(phiVc2)
    
    phiVc3 = phi*0.27*(2+((alphas*de)/bo))*lmbda*np.sqrt(fprimc)*bw*de /1000    #ACI 22.6.5.2 (c)
    phiVc3mat.append(phiVc3)

    phiVc2011 = phi*(1.06*np.sqrt(fprimc)*bw*de) /1000
    phiVc2011mat.append(phiVc2011)

    phiVcmax = phi*1.33*lmbda*np.sqrt(fprimc)*bw*de /1000    #ACI 22.5.5.1.1
    phiVcmaxmat.append(phiVcmax)
    

    phiVcd2 = np.min([phiVc1,phiVc2,phiVc3,phiVcmax])   #Resultado final tomando el menor de los tres
    phiVcd2mat.append(phiVcd2)

    phiVcd = np.min([phiVc2011,phiVcmax])       #Resultado considerando la norma del 2011
    phiVcdmat.append(phiVcd)

    Cortanteactual.append(phiVcd2)
    Cortante2011.append(phiVcd)
    dval.append(de)

    ratiod =  Vu1/phiVcd
    ratiod2 = Vu2/phiVcd2
    ratiodmat.append(ratiod)
    ratiod2mat.append(ratiod2)
 
    i+=1
    de+=5


df = pd.DataFrame([[dval,lmbdas,bomat,Vu1mat,Vu2mat,phiVc1mat,phiVc2mat,phiVc3mat,phiVc2011mat,phiVcmaxmat,phiVcdmat,phiVcd2mat,ratiodmat,ratiod2mat]],
columns=['d En cm','Lambda_s','bo','Vu1 En ton@m','Vu2 En ton@m','phiVc1 En ton@m','phiVc2 En ton@m','phiVc3 En ton@m','phiVc2011 En ton@m','phiVcmax En ton@m','phiVcd En ton@m','phiVcd2 En ton@m','Ratio d','Ratio 2d'])
df.style



,d En cm,Lambda_s,bo,Vu1 En ton@m,Vu2 En ton@m,phiVc1 En ton@m,phiVc2 En ton@m,phiVc3 En ton@m,phiVc2011 En ton@m,phiVcmax En ton@m,phiVcd En ton@m,phiVcd2 En ton@m,Ratio d,Ratio 2d
0,[57.5],0.778499,[530.0],[16],[18],[56.00718424898721],[68.25688390558386],[111.94899471608636],[69.32643874724131],[86.98505993757635],[69.32643874724131],[56.00718424898721],[0.23079218100809604],[0.32138734059506835]


<p>Recubrimiento:</p>
<img src='./images/recubrimiento.jpg'>

<p>Chequeo de flexion</p>
<img src='./images/formula1.jpg'>

In [12]:
#Calculo del peso propio del plinto
Ag = dval[-1] * bw
Asmin = 0.0018 * Ag # Obtenido en ACI 8.6.1.1
h = dval[-1] + rec + 2.54 #Recubrimiento indicado en la ACI+ el dametro de una varilla de soporte (25.4mm asumidos)
Pp = (h*A0/1000000) * 2.4  #Peso de la zapata (2.4 T/m3 peso especifico del hormigon armado) & Recubrimiento basado en tabla 20.5.1.3.1 (ACI)
Pcuello = (cc*cl*100/1000000) * 2.4 #Peso del cuello de la columna
Prelleno = ((A0*100/1000000) - (cc*cl*100/1000000)) * 1.8 #Asumimos que el suelo posee un peso específico de 1.5 T/m3
Ptot = (Pp + Pcuello + Prelleno + (Psu/1000)) * 1000
qo = (Ptot / A0)


print(f'El valor de h es: {h} cm')
print(f'El valor de la carga de la zapata es: {Pp} tons')
print(f'El valor de la carga del cuello es: {Pcuello} tons')
print(f'El valor de la carga del relleno es: {Prelleno} tons')
print(f'El valor del peso total es: {Ptot} kg')
print(f'El valor del esfuerzo de contacto es: {qo} kg/cm2')

if qo <= qa:
    print(f'El tamaño del plinto cumple, qo: {qo} <= qa: {qa} ')
else:
    print(f'\nEl tamaño de la zapata no cumple,  qo: {qo} <= qa: {qa} (en kg/cm2)')
    while qo > qa:
        qo = Ptot / A0
        A0+=100 #cm2 
    
    Lmax = np.sqrt(A0)
    print(f'La nueva dimension de la zapata es {Lmax} cm')

    #Verificacion de espesor minimo requerido por flexión
    c = ((Lmax/2)-(np.max([cc,cl])/2))
    Mu = (qo*Lmax*(c**2))/2

    Rn = (Mu)/(0.9*bw*(dval[-1]**2)) #Ver formula en referencias
    ro = (0.85*fprimc/fy)*(1-np.sqrt(1-((2*Rn)/(0.85*fprimc))))
    As = ro*bw*dval[-1]

    if As <= Asmin:
        print(f'\nEl acero requerido As <=> Asmin: {Asmin} cm2 >= As: {As} cm2')
    else:
        print(f'\nEl acero requerido es As: {As} cm2 >= Asmin: {Asmin} cm2')

    #Calculo de phiMn con este Asmin de refuerzo:
    a = (Asmin*fy) / (0.85*fprimc*bw)
    phiMn = (0.9*Asmin*fy*(dval[-1]-(a/2)))*(Lmax/bw)

    if phiMn >= Mu:
        print(f'La reistencia de Mn con Asmin cumple la demanda, phiMn:{phiMn /100000} >= Mu:{Mu /100000} T*m@m')
    else:
        print(f'La reistencia de Mn con Asmin no cumple la demanda, phiMn:{phiMn /100000} < Mu:{Mu /100000} T*m@m')


# ro = ((Asmin/0.9) / (bw*dval[0])) #Se divide con 0.9 porque la formula considera con H sino con d.(Se asume que d/H = 0.9)
# defl = (2.2*c)*np.sqrt(Ptot/A0) #Siendo delf el d a flexion, obtenido despejando, ver ecuacion 10.16 del (DGboACI)
#print(f'El valor de d (comparando capacidad a flexion y corte) es: {np.max([defl,dval[0]])} cm')

El valor de h es: 67.54 cm
El valor de la carga de la zapata es: 4.4130636 tons
El valor de la carga del cuello es: 1.3259999999999998 tons
El valor de la carga del relleno es: 3.906 tons
El valor del peso total es: 53165.0636 kg
El valor del esfuerzo de contacto es: 1.9528030707070707 kg/cm2
El tamaño del plinto cumple, qo: 1.9528030707070707 <= qa: 2 


In [13]:
#Numero de varillas y espaciamiento siendo varillas de 14mm
Asv = np.pi*(1.4**2)*0.25
numvar = np.round(Asmin/Asv)
espaciamiento = (bw)/(numvar)
print(f'El numero de varillas requerido es: {numvar}')
print(f'El espaciamiento entre varillas es: {espaciamiento} cm')

El numero de varillas requerido es: 7.0
El espaciamiento entre varillas es: 14.285714285714286 cm
